In [1]:
import pandas as pd
import geopandas as gpd
import h3
import os
import matplotlib.pyplot as plt
import networkx as nx
import osmnx as ox
# !pip install descartes
import descartes
from descartes import PolygonPatch
from shapely.geometry import Point, LineString, Polygon

In [2]:
# import poi_pop_h3 file
poi_pop_h3 = gpd.read_file(os.path.join('data','cleaned','poi_pop_h3_r10.geojson'))
# import exit flow data
flow_bus = gpd.read_file(os.path.join('data','cleaned','flow_bus_byStop.geojson'))
flow_rail = gpd.read_file(os.path.join('data','cleaned','flow_rail.geojson'))

# Preparations for isochrones

Prep the network

In [3]:
## Takes 12 minutes

# load the street network in Greater London
G = ox.graph_from_place('Greater London, UK', network_type='walk')
G = ox.project_graph(G, to_crs='EPSG:27700') # Project graph to British National Grid

# export the graph to local as graphml
ox.io.save_graphml(G, filepath=os.path.join('data','cleaned','G.graphml'))

In [4]:
# load graph from graphml
G = ox.io.load_graphml(os.path.join('data','cleaned','G.graphml'))

In [5]:
# plot the graph
fig, ax = ox.plot_graph(G, node_size=0, edge_linewidth=0.5, bgcolor='w', show=False, close=False)

Functions to find isochrone from single point

In [6]:
# Function to get isochrone polygons from a center node on a graph
def make_iso_polys(G, center_node, edge_buff=25, node_buff=50):
    # isochrone_polys = []
    for trip_time in sorted(trip_times, reverse=True):
        subgraph = nx.ego_graph(G, center_node, radius=trip_time, distance='time')

        node_points = [Point((data['x'], data['y'])) for node, data in subgraph.nodes(data=True)]
        nodes_gdf = gpd.GeoDataFrame({'id': subgraph.nodes()}, geometry=node_points)
        nodes_gdf = nodes_gdf.set_index('id')

        edge_lines = []
        for n_fr, n_to in subgraph.edges():
            f = nodes_gdf.loc[n_fr].geometry
            t = nodes_gdf.loc[n_to].geometry
            edge_lookup = G.get_edge_data(n_fr, n_to)[0].get('geometry',  LineString([f,t]))
            edge_lines.append(edge_lookup)

        n = nodes_gdf.buffer(node_buff).geometry
        e = gpd.GeoSeries(edge_lines).buffer(edge_buff).geometry
        all_gs = list(n) + list(e)
        isochrone_polys = gpd.GeoSeries(all_gs).unary_union
    
    return isochrone_polys

# Wrapper Function to get isochrone from a point geometry
def get_isochrone_walk(point):
    
    global G
    x, y = point.x, point.y # Define point coords
    center_node = ox.nearest_nodes(G, x, y) # Find center node

    # get the isochrone polygons
    isochrone_polys = make_iso_polys(G, center_node, edge_buff=25, node_buff=0)
    
    return isochrone_polys

# Wrapper Function to plot isochrone
def plot_isochrone(point):

    global G
    x, y = point.x, point.y # Define point coords
    center_node = ox.nearest_nodes(G, x, y) # Find center node

    # get the isochrone polygons
    isochrone_polys = make_iso_polys(G, center_node, edge_buff=25, node_buff=0)

    # plot it
    iso_colors = ox.plot.get_colors(n=len(trip_times), cmap='plasma', start=0)
    fig, ax = ox.plot_graph(G, show=False, close=False, edge_color='#999999', edge_alpha=0.2,node_size=0, bgcolor='k')
    for polygon, fc in zip(isochrone_polys, iso_colors):
        patch = PolygonPatch(polygon, fc=fc, ec='none', alpha=0.6, zorder=-1)
        ax.add_patch(patch)
    plt.show()

# ISOCHRONE FOR RAIL

In [7]:
# import exit flow data
flow_rail = gpd.read_file(os.path.join('data','cleaned','flow_rail.geojson'))

In [8]:
%%time 

# configure the place, network type, trip times, and travel speed
network_type = 'walk'
trip_times = [10] #in minutes
travel_speed = 5 #walking speed in km/hour

# takes a while to run 4.6/stop
# add isochrone polygons to flow_rail
flow_rail['isochrone'] = flow_rail.geometry.apply(get_isochrone_walk)
flow_rail = gpd.GeoDataFrame(flow_rail, geometry='isochrone', crs='EPSG:27700').drop(columns='geometry') # convert to geodataframe and drop the old geometry column

In [ ]:
flow_rail.to_file(os.path.join('data','cleaned','rail_isochrone.gpkg'), driver='GPKG')

# ISOCHRONES FOR BUS

In [10]:
# import exit flow data (Bus)
flow_bus = gpd.read_file(os.path.join('data','cleaned','flow_bus_byStop.geojson'), driver='GeoJSON', crs='EPSG:27700')
flow_bus.explore()

In [11]:
# split the flow_bus dataset into chunks of 1000 rows
flow_bus_chunks = [flow_bus[i:i+1000] for i in range(0, flow_bus.shape[0], 1000)]

# configure the place, network type, trip times, and travel speed
network_type = 'walk'
trip_times = [10] #in minutes
travel_speed = 5 #walking speed in km/hour

for i, chunk in enumerate(flow_bus_chunks):
    chunk['isochrone'] = chunk.geometry.apply(get_isochrone_walk)
    chunk = gpd.GeoDataFrame(chunk, geometry='isochrone', crs='EPSG:27700').drop(columns='geometry') # convert to geodataframe and drop the old geometry column
    chunk.to_file(os.path.join('data','cleaned','bus_isochrone','bus_isochrone_{}.geojson'.format(i)), driver='GeoJSON')

In [12]:
# import all the flow_bus_isochrone datasets from files into a list
bus_isochrone_list = []
for i in range(0, 20):
    bus_isochrone_list.append(gpd.read_file(os.path.join('data','cleaned','bus_isochrone','bus_isochrone_{}.geojson'.format(i))))

# concatenate all the flow_bus_isochrone datasets into one
bus_isochrone = pd.concat(bus_isochrone_list)
bus_isochrone.reset_index(inplace=True,drop=True)
bus_isochrone.info()

In [13]:
# save the flow_bus_isochrone dataset to fileb
bus_isochrone.to_file(os.path.join('data','cleaned','bus_isochrone.gpkg'), driver='GPKG')